# Meiro Data Analyst Engineer Task

### Data processing
---

In [1]:
# Importing the required libraries
import pandas as pd
import plotly.express as px
import sqlite3

In [2]:
# Read the .ndjson file into a pandas DataFrame
df = pd.read_json(path_or_buf="./data/data.ndjson.gz", compression="gzip", lines=True)

# Updating specific column names and data types
df.rename(columns={"id": "order_id", "created": "order_created"}, inplace=True)
df["order_created"] = pd.to_datetime(df["order_created"], unit="s")

# Adding additional date dimensions for analysis
df["order_year"] = df["order_created"].dt.year
df["order_month"] = df["order_created"].dt.month.astype(str).str.zfill(2)
df["order_day"] = df["order_created"].dt.day
df["order_day_of_week"] = df["order_created"].dt.day_name()

df["order_created"] = df["order_created"].dt.date

print(df.shape)
df.head()

(100000, 8)


,order_id,order_created,products,user,order_year,order_month,order_day,order_day_of_week
0,0,2018-11-16,"[{'id': 0, 'name': 'Product A', 'price': 160},...","{'id': 3, 'name': 'User D', 'city': 'Sydney'}",2018,11,16,Friday
1,1,2018-12-06,"[{'id': 3, 'name': 'Product D', 'price': 130},...","{'id': 0, 'name': 'User A', 'city': 'Prague'}",2018,12,6,Thursday
2,2,2018-10-21,"[{'id': 6, 'name': 'Product G', 'price': 100},...","{'id': 0, 'name': 'User A', 'city': 'Prague'}",2018,10,21,Sunday
3,3,2018-11-12,"[{'id': 6, 'name': 'Product G', 'price': 100},...","{'id': 2, 'name': 'User C', 'city': 'Singapore'}",2018,11,12,Monday
4,4,2018-11-30,"[{'id': 10, 'name': 'Product K', 'price': 60},...","{'id': 5, 'name': 'User F', 'city': 'Jakarta'}",2018,11,30,Friday


In [3]:
# Splitting the "user" information into Tabular format & updating column names
user_df = pd.json_normalize(df["user"])
user_df.columns = [f"user_{col}" for col in user_df.columns]

# TEST: Checking if the number of rows in the user_df matches with the original df
assert len(user_df) == len(df), "rows in user_df do not match with df"

# Joining the user columns with the original df and dropping the redundant column
df = df.join(user_df)
df.drop(columns=["user"], inplace=True)

print(df.shape)
df.head()

(100000, 10)


,order_id,order_created,products,order_year,order_month,order_day,order_day_of_week,user_id,user_name,user_city
0,0,2018-11-16,"[{'id': 0, 'name': 'Product A', 'price': 160},...",2018,11,16,Friday,3,User D,Sydney
1,1,2018-12-06,"[{'id': 3, 'name': 'Product D', 'price': 130},...",2018,12,6,Thursday,0,User A,Prague
2,2,2018-10-21,"[{'id': 6, 'name': 'Product G', 'price': 100},...",2018,10,21,Sunday,0,User A,Prague
3,3,2018-11-12,"[{'id': 6, 'name': 'Product G', 'price': 100},...",2018,11,12,Monday,2,User C,Singapore
4,4,2018-11-30,"[{'id': 10, 'name': 'Product K', 'price': 60},...",2018,11,30,Friday,5,User F,Jakarta


In [4]:
# Transforming each product list item in the "products" column into a separate row
orders_df = pd.DataFrame(df["products"].explode())

# Storing the index of the original df for future joins
orders_index = orders_df.index.to_list()

print(orders_df.shape)
orders_df.head()

(549145, 1)


,products
0,"{'id': 0, 'name': 'Product A', 'price': 160}"
0,"{'id': 8, 'name': 'Product I', 'price': 80}"
1,"{'id': 3, 'name': 'Product D', 'price': 130}"
1,"{'id': 2, 'name': 'Product C', 'price': 140}"
1,"{'id': 13, 'name': 'Product N', 'price': 30}"


In [5]:
# Splitting the "products" information into Tabular format & updating column names
orders_df = pd.json_normalize(orders_df["products"])
orders_df.columns = [f"product_{col}" for col in orders_df.columns]

# Replacing index with the original index representative of the orders in the original df
orders_df["index"] = orders_index
orders_df.set_index("index", inplace=True)

print(orders_df.shape)
orders_df.head()

(549145, 3)


,product_id,product_name,product_price
index,,,
0,0,Product A,160
0,8,Product I,80
1,3,Product D,130
1,2,Product C,140
1,13,Product N,30


In [6]:
# Creating one master table with all the order, product, user information and dropping the redundant column
orders_df = orders_df.join(df.drop(columns=["products"])).reset_index(drop=True)

print(orders_df.shape)
orders_df.head()

(549145, 12)


,product_id,product_name,product_price,order_id,order_created,order_year,order_month,order_day,order_day_of_week,user_id,user_name,user_city
0,0,Product A,160,0,2018-11-16,2018,11,16,Friday,3,User D,Sydney
1,8,Product I,80,0,2018-11-16,2018,11,16,Friday,3,User D,Sydney
2,3,Product D,130,1,2018-12-06,2018,12,6,Thursday,0,User A,Prague
3,2,Product C,140,1,2018-12-06,2018,12,6,Thursday,0,User A,Prague
4,13,Product N,30,1,2018-12-06,2018,12,6,Thursday,0,User A,Prague


### Star schema data model
---
- `fact_order`: The central _fact_ table in this star schema captures details about orders and has foreign keys linking to `dim_user`:
  - order_id (PK): The primary key that uniquely identifies each order.
  - user_id (FK1): A foreign key linking to dim_user.
  - order_created, order_year, order_month, order_day, order_day_of_week: Descriptive attribute - Date-related fields that break down the order creation date into its components.
  - order_total: A measure that typically represents the total amount of the order.
- `dim_user`: A _dimension_ table that holds information about the users. It is connected to the `fact_order` table:
  - user_id (PK): The primary key that uniquely identifies each user.
  - user_name: Descriptive attribute - The name of the user.
  - user_city: Descriptive attribute - The city where the user is located.
- `bridge_order_lineitems` (_junction_ table): This table connects `fact_order` and `dim_product`, facilitating a many-to-many relationship between orders and products. Each row in this table represents a line item in an order:
  - order_lineitem_id (PK): A primary key that uniquely identifies each line item in an order.
  - order_id (FK1): A foreign key that references `fact_order`.
  - product_id (FK2): A foreign key that references `dim_product`.
- `dim_product`: A _dimension_ table with details on products. It stands alone in this schema with no direct relationships depicted to the `fact_order` table:
  - product_id (PK): The primary key uniquely identifying each product.
  - product_name: Descriptive attribute - The name of the product.
  - product_price: Descriptive attribute - The price of the product.

The relationships depicted are:
- `fact_order` to `dim_user` to: A many-to-one relationship, as one user can have many orders, but each order is associated with only one user.
- `fact_order` to `bridge_order_lineitems`: A one-to-many relationship, as one order can have many line items purchased, but each line item is associated with only one order.
- `bridge_order_lineitems` to `dim_product`: A many-to-one relationship, as one product can appear in many line items, but each line item references one product.

![Star schema data model diagram](./datamodel_starschema.png)
---

In [7]:
# Creating first Dimensional table from master table with User information
dim_user = (
    orders_df.groupby(["user_id", "user_name", "user_city"])[["order_id"]]
    .count()
    .reset_index()
    .drop(columns=["order_id"])
)

# TEST: Checking if the users are unique
assert dim_user["user_id"].nunique() == len(dim_user), "user id is not primary key"

print(dim_user.shape)
dim_user.head()

(8, 3)


,user_id,user_name,user_city
0,0,User A,Prague
1,1,User B,Brno
2,2,User C,Singapore
3,3,User D,Sydney
4,4,User E,Melbourne


In [8]:
# Creating second Dimensional table from master table with Product information
dim_product = (
    orders_df.groupby(["product_id", "product_name", "product_price"])[["order_id"]]
    .count()
    .reset_index()
    .drop(columns=["order_id"])
)

# TEST: Checking if the products are unique
assert dim_product["product_id"].nunique() == len(
    dim_product
), "prod id is not primary key"

print(dim_product.shape)
dim_product.head()

(16, 3)


,product_id,product_name,product_price
0,0,Product A,160
1,1,Product B,150
2,2,Product C,140
3,3,Product D,130
4,4,Product E,120


In [9]:
# Creating Fact table from master table with Order information
fact_order = (
    orders_df.groupby(
        [
            "order_id",
            "order_created",
            "order_year",
            "order_month",
            "order_day",
            "order_day_of_week",
            "user_id",
        ]
    )[["product_price"]]
    .sum()
    .reset_index()
)

# Renaming columns to be more representative
fact_order.rename(columns={"product_price": "order_total"}, inplace=True)

# TEST: Checking if the orders are unique
assert fact_order["order_id"].nunique() == len(
    fact_order
), "order id is not primary key"

print(fact_order.shape)
fact_order.head()

(100000, 8)


,order_id,order_created,order_year,order_month,order_day,order_day_of_week,user_id,order_total
0,0,2018-11-16,2018,11,16,Friday,3,240
1,1,2018-12-06,2018,12,6,Thursday,0,300
2,2,2018-10-21,2018,10,21,Sunday,0,190
3,3,2018-11-12,2018,11,12,Monday,2,680
4,4,2018-11-30,2018,11,30,Friday,5,370


In [10]:
# Creating Bridge table from master table with Order and Product information
bridge_order_lineitems = orders_df[["order_id", "product_id"]].copy()

bridge_order_lineitems["order_lineitem_id"] = (
    bridge_order_lineitems.groupby("order_id").cumcount() + 1
)
bridge_order_lineitems["order_lineitem_id"] = bridge_order_lineitems["order_id"].astype(
    str
) + bridge_order_lineitems["order_lineitem_id"].astype(str)

# TEST: Checking if the order line items are unique
assert bridge_order_lineitems["order_lineitem_id"].nunique() == len(
    bridge_order_lineitems
), "line item id is not primary key"

print(bridge_order_lineitems.shape)
bridge_order_lineitems.head()

(549145, 3)


,order_id,product_id,order_lineitem_id
0,0,0,01
1,0,8,02
2,1,3,11
3,1,2,12
4,1,13,13


In [11]:
# Exporting the Dimensional and Fact tables to pkl files for Streamlit app
dim_user.to_pickle("./data/dim_user.pkl")
dim_product.to_pickle("./data/dim_product.pkl")
fact_order.to_pickle("./data/fact_order.pkl")
bridge_order_lineitems.to_pickle("./data/bridge_order_lineitems.pkl")

### SQLite DB creation
---

In [12]:
# Creating a new SQLite DB file
con = sqlite3.connect("dwh.db")

In [13]:
# Storing the Fact and Dimensional table DataFrames in the SQLite DB
dim_user.to_sql(name="dim_user", con=con, index=False, if_exists="replace")
dim_product.to_sql(name="dim_product", con=con, index=False, if_exists="replace")
fact_order.to_sql(name="fact_order", con=con, index=False, if_exists="replace")
bridge_order_lineitems.to_sql(
    name="bridge_order_lineitems", con=con, index=False, if_exists="replace"
)

549145

#### Answer Queries
---

> Question 1: Which user spent the most money on products on all Fridays?

In [14]:
sql_query1 = """
WITH friday_spend AS (
    SELECT user_id, sum(order_total) as total_friday_spend
    FROM fact_order
    WHERE order_day_of_week = 'Friday'
    GROUP BY user_id
)

SELECT friday_spend.user_id, dim_user.user_name, dim_user.user_city, friday_spend.total_friday_spend
FROM friday_spend
JOIN dim_user
ON friday_spend.user_id = dim_user.user_id
ORDER BY friday_spend.total_friday_spend DESC
LIMIT 1;
"""
sql_df1 = pd.read_sql(sql_query1, con)
sql_df1

,user_id,user_name,user_city,total_friday_spend
0,4,User E,Melbourne,874560


> Answer to Question 1: The user with maximum Friday spend is **User E from Melbourne** with a total spend of **~$874k**.

> Question 2: What are the best 3 products in each location of a user based on quantity?

In [15]:
sql_query2 = """
WITH prod_city AS (
    SELECT items.order_id, items.product_id, fact_order.user_id, dim_user.user_city, dim_product.product_name
    FROM bridge_order_lineitems as items
    JOIN fact_order
    ON items.order_id = fact_order.order_id
    JOIN dim_user
    ON fact_order.user_id = dim_user.user_id
    JOIN dim_product
    ON items.product_id = dim_product.product_id
),
prod_count AS (
    SELECT user_city, product_name, count(product_name) as product_count
    FROM prod_city
    GROUP BY user_city, product_name
),
ranked_products AS (
    SELECT user_city, product_name, product_count,
    ROW_NUMBER() OVER(PARTITION BY user_city ORDER BY product_count DESC) as rank
    FROM prod_count
)
SELECT user_city, product_name, product_count, rank
FROM ranked_products
WHERE rank <= 3
"""
sql_df2 = pd.read_sql(sql_query2, con)
sql_df2

,user_city,product_name,product_count,rank
0,Brno,Product L,4438,1
1,Brno,Product O,4413,2
2,Brno,Product N,4386,3
3,Hong Kong,Product I,4415,1
4,Hong Kong,Product P,4397,2
5,Hong Kong,Product O,4369,3
6,Jakarta,Product D,4328,1
7,Jakarta,Product J,4309,2
8,Jakarta,Product C,4305,3
9,Kuala Lumpur,Product B,4446,1


> Answer to Question 2: Above output captures top 3 best products per user city location based on purchased quantity.

In [16]:
# Exporting the Query outputs to pkl files for Streamlit app
sql_df1.to_pickle("./data/sql_df1.pkl")
sql_df2.to_pickle("./data/sql_df2.pkl")

### Additional Insights
---

#### 1. Top Sales Months

In [17]:
# Creating a line plot to visualize the total sales over time
date_grouped_df = fact_order.groupby("order_created")["order_total"].sum().reset_index()

fig1 = px.line(
    date_grouped_df,
    x="order_created",
    y="order_total",
    title="Sales Trend over Time",
    width=1000,
)
fig1.show("png")

In [18]:
# Filtering only to September 2018 transactions
sept_df = fact_order[fact_order["order_month"] == "09"].reset_index(drop=True)

# Creating a bar chart to visualize the sales trend in September 2018
fig2 = px.bar(
    sept_df.groupby("order_day")["order_total"].sum().reset_index(),
    x="order_day",
    y="order_total",
    title="Sales Trend in September 2018",
    width=1000,
)
fig2.show("png")

In [19]:
# Filtering only to December 2018 transactions
dec_df = fact_order[fact_order["order_month"] == "12"].reset_index(drop=True)

# Creating a bar chart to visualize the sales trend in September 2018
fig3 = px.bar(
    dec_df.groupby("order_day")["order_total"].sum().reset_index(),
    x="order_day",
    y="order_total",
    title="Sales Trend in December 2018",
    width=1000,
)
fig3.show("png")

In [20]:
# Creating a line plot to visualize the total sales over time excluding the first and last date with outlier values
date_grouped_df = fact_order.groupby("order_created")["order_total"].sum().reset_index()

fig4 = px.line(
    date_grouped_df.iloc[1:-1],
    x="order_created",
    y="order_total",
    title="Sales Trend over Time",
    width=1000,
)
fig4.show("png")

In [21]:
# Yearly sales counts
fact_order["order_year"].value_counts().sort_values(ascending=False)

2018    100000
Name: order_year, dtype: int64

In [22]:
# Monthly sales counts
fact_order["order_month"].value_counts().sort_values(ascending=False)

10    34716
11    33213
09    22386
12     9685
Name: order_month, dtype: int64

In [23]:
# Monthly sales revenue
fact_order.groupby("order_month")["order_total"].sum().sort_values(ascending=False)

order_month
10    16157610
11    15504140
09    10445020
12     4553890
Name: order_total, dtype: int64

  **Insight:**
  - The dataset includes sales transactions from September 2018 to December 2018.
  - Analysis suggests spike in sales during end **October**, possibly due to seasonal promotional event.
  - Possible assumption for reduced sales on the 1st and last day (10 Sept and 9 Dec respectively) in the given dataset could be lack of capture of all transaction records.

#### 2. Top 5 Spenders

In [24]:
# Creating a bar plot to visualize the top 5 User spenders
user_grouped_df = (
    fact_order.merge(dim_user, on="user_id")
    .groupby("user_name")["order_total"]
    .sum()
    .reset_index()
    .sort_values("order_total", ascending=False)
    .head()
)

fig5 = px.bar(
    user_grouped_df,
    y="user_name",
    x="order_total",
    title="Top Sales By User",
    orientation="h",
    width=1000,
).update_yaxes(type="category", categoryorder="max ascending")
fig5.show("png")

  **Insight:**
  - **User B** is the top spender during the captured period in the dataset with a total spend of ~**$5.9M**.

#### 3. Sales by User Locations

In [25]:
# Creating a pie plot to visualize the total sales by User city
user_city_sales = (
    fact_order.merge(dim_user, on="user_id")
    .groupby("user_city")["order_total"]
    .sum()
    .reset_index()
)

fig6 = px.pie(
    user_city_sales,
    values="order_total",
    names="user_city",
    title="Total Sales By City",
    width=1000,
)
fig6.show("png")

  **Insight:**
  - The sales revenue is almost **equally distributed** across all the user geographies.  

---